## Initialisations

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import model_selection

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn import datasets

In [ ]:
from keras.datasets import mnist

from keras.models import Sequential, load_model

from keras.layers import Dense, Dropout, Flatten

from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.utils.np_utils import to_categorical

## Le dataset MNIST du langage des signes

Question 1

On utilise le dataset du langage des signes :  
https://www.kaggle.com/datamunge/sign-language-mnist#american_sign_language.PNG

In [ ]:
df=pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')

In [ ]:
df.shape

27455 images de 784 pixels

In [ ]:
df.head()

Le dataset est bien équilibré (mais il manque les lettres Y et Z) :

In [ ]:
df.label.value_counts()

In [ ]:
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
print(labels)

On convertit les lignes de pixels en matrices (images) :

In [ ]:
n_samples = len(df.index)
images = np.array(df.drop(['label'],axis=1))
images = images.reshape(n_samples,28,28)   #images[i] : liste de liste contenant les valeurs des pixels

On affiche les 50 premiers :

In [ ]:
plt.figure(figsize=(10,20))
for i in range(0,49) :
    plt.subplot(10,5,i+1)
    plt.axis('off')
    plt.imshow(images[i], cmap="gray_r")
    plt.title(labels[df.label[i]])

On sépare la cible et les caractéristiques :

In [ ]:
y = df['label']
X = df.drop(['label'] , axis=1)

On normalise les valeurs entre 0 et 1 :

In [ ]:
X = X/255

Séparation train / test :

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [ ]:
print(len(images))

Il y a bien 28 pixels

Comme l'activation d'un neurone donne une valeur (probabilité) entre 0 ou 1, on code la cible (classes entre 0 et 24) sous la forme d'un vecteur de 0 ou 1 (*one hot encoding*) avec *to_categorical* :

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
y_cat = to_categorical(y)
print(y_cat[0])

On a seulement 25 classes (il manque les Y et Z dans le dataset) :

In [ ]:
num_classes = y_cat.shape[1]
print(num_classes)

On sépare train et test :

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=1)

In [ ]:
print(len(y_test))

Pour Keras, il est nécessaire d'avoir des tableaux et non des dataframes :

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
X_train=X_train.reshape(len(X_train),28,28,1)
X_test=X_test.reshape(len(X_test),28,28,1)

In [ ]:
print(X_train.shape)  #X_train[i]=image X_train[i][j][k]=pixel ligne j colonne k de l'image i
print(y_train.shape)
print(X_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Modèle CNN sans dropout
model1 = Sequential()
model1.add(Conv2D(64, (3, 3), input_shape=(28, 28, 1), activation='relu'))  #premiere couche (on précise input shape)
model1.add(Conv2D(64, (3, 3), activation='relu'))                           #deuxième couche
model1.add(MaxPooling2D(pool_size=(2, 2)))                                  #on divise l'image par deux

model1.add(Conv2D(20, (3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Flatten())                         #j'aplatis
model1.add(Dense(512, activation='relu'))     #je remet une couche dense de 512 neurones
model1.add(Dense(num_classes, activation='softmax'))                      #couche fully-connected pour identifier les classes



# Compilation du modèle
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model1.summary()

Question 2

In [ ]:
# Apprentissage
train = model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)

relu souvent plus efficace que la sinusoide où le gradient peut être très faible->on met bcp de temps pr converger.

On "compile" le modèle, avec une *categorical_crossentropy* comme mesure de distance (distance probabiliste multi classes)

méthode adam permet d'accélérer la descente du gradient

Question 3

In [ ]:
model1.evaluate(X_test,y_test)

La variable *train* mémorise l'historique des scores sur l'ensemble d'apprentissage :

In [ ]:
print(train.history['accuracy'])

et sur l'ensemble de validation :

In [ ]:
print(train.history['val_accuracy'])

On définit une fonction pour afficher un graphique des scores :

In [ ]:
def plot_scores(train) :
    accuracy = train.history['accuracy']
    val_accuracy = train.history['val_accuracy']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='Score apprentissage')
    plt.plot(epochs, val_accuracy, 'r', label='Score validation')
    plt.title('Scores')
    plt.legend()
    plt.show()

In [ ]:
plot_scores(train)

In [ ]:
# Modèle CNN avec dropout
model2 = Sequential()
model2.add(Conv2D(64, (3, 3), input_shape=(28, 28, 1), activation='relu'))  #premiere couche (on précise input shape)
model2.add(Conv2D(64, (3, 3), activation='relu'))                           #deuxième couche
model2.add(MaxPooling2D(pool_size=(2, 2)))                                  #on divise l'image par deux
model2.add(Dropout(0.1))                                                    #10% des neurones tirés au sort n'apprennent pas, peut réduire le surapprentissage
model2.add(Conv2D(20, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.1))
model2.add(Flatten())                         #j'aplatis
model2.add(Dense(512, activation='relu'))     #je remet une couche dense de 512 neurones
model2.add(Dense(num_classes, activation='softmax'))                      #couche fully-connected pour identifier les classes



# Compilation du modèle
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Apprentissage
train = model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)

In [ ]:
model2.evaluate(X_test,y_test)

In [ ]:
print(train.history['accuracy'])

In [ ]:
print(train.history['val_accuracy'])

In [ ]:
plot_scores(train)

Question 4

In [ ]:
import random
import math
import numpy

In [ ]:
print(X_test.shape)

In [ ]:
plt.figure(figsize=(10,20))
for i in range(0,49) :
    plt.subplot(10,5,i+1)
    plt.axis('off')
    plt.imshow(images[i], cmap="gray_r")
    plt.title(labels[df.label[i]])

In [ ]:
sigma=10
for k in range(len(X_test)):                #différentes images
    for j in range(len(X_test[0])):        #ligne des pixels     
        for i in range(len(X_test[0][0])):  #colonne
            v = int(math.floor(X_test[k][j][i][0]+random.gauss(0,sigma)))
            if v > 255:
                v = 255
            if v<0:
                v = 0
            X_test[k][j][i][0] = v

In [ ]:
images=X_test.reshape(len(X_test),28,28)

In [ ]:
plt.figure(figsize=(10,20))
for i in range(0,49) :
    plt.subplot(10,5,i+1)
    plt.axis('off')
    plt.imshow(images[i], cmap="gray_r")
    plt.title(labels[df.label[i]])

Question 5

In [ ]:
# Apprentissage
train = model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)

In [ ]:
y_model1 = model1.predict(X_test)

In [ ]:
#print(y_model1)

In [ ]:
#model1.evaluate(X_test,y_test)

In [ ]:
#print(train.history['accuracy'])

In [ ]:
#print(train.history['val_accuracy'])

In [ ]:
#plot_scores(train)

In [ ]:
Y_model2=[]
for i in range(10):
    train = model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)
    y_model2 = model2.predict(X_test)
    Y_model2.append(y_model2)

    

In [ ]:
Y_model2[i][j][k]


In [ ]:
from math import *
y_model2=[]
for i in range(len(Y_model2[0])):
    M=[]
    for j in range(len(Y_model2[0][0])):     #pour chaque prédiction de chaque image
        M.append(0)
    y_model2.append(M)
        



for j in range(len(Y_model2[0])):   #[[0,1,0,0,0,0,0],[0,0,0,1,0,0,0]]
    for k in range(len(Y_model2[0][0])):              #pour chaque nombre binaire correspondant à la prédiction de l'image pour chaque image
        s=0
        for i in range(len(Y_model2)):                #pour chaque essai de prédiction
            s=s+float(Y_model2[i][j][k])
        s=s/10
        s=round(s)
        y_model2[j][k]=s


In [ ]:
print(accuracy_score(y_test,y_model1))
print(accuracy_score(y_test,y_model2))